In [2]:
import pandas as pd
import pymysql
import datetime

In [3]:
conn = pymysql.connect(host='rr-wz9y03t9wtq075179o.mysql.rds.aliyuncs.com', user='medbuser', passwd='Ptmesys!!!1290098', db='mobility_exchange', port = 3306)
sql = 'SELECT TIME(FROM_TIME) as TIME,{} FROM me_transport_hub_minute_stat WHERE HUB_CODE="SZ_AIR_BA" AND DATE(FROM_TIME)="{}" ORDER BY CRT_ON;'

In [10]:
def getTrafficHubData(fieldName, days, resultFileName=""):
    field_df = pd.DataFrame()
    date_today = datetime.date.today()
    if len(resultFileName)==0:
        resultFileName="{}_{}.csv".format(fieldName,date_today)
        
    for i in range(days):
        date_i = date_today - datetime.timedelta(days=(i+1))
        sql_i = sql.format(fieldName,date_i)
        df_i = pd.read_sql_query(sql_i, con=conn)
        tmp_df = pd.DataFrame()
        
        if len(field_df.columns)==0:
            field_df[[date_i,"TIME"]] = df_i[[fieldName,"TIME"]]
        else:
            tmp_df[[date_i, "TIME"]] = df_i[[fieldName, "TIME"]]
            field_df = pd.merge(field_df, tmp_df, how="outer", on="TIME")
    field_df.drop_duplicates(inplace=True)
    field_df.reset_index(drop=True,inplace=True)
    del field_df["TIME"]
    field_df.to_csv(resultFileName)
    return field_df    

In [11]:
df = getTrafficHubData(days=10,fieldName="VEHICLE_COUNT")

In [12]:
df.describe()

,2018-07-04,TIME,2018-07-03,2018-07-02,2018-07-01,2018-06-30,2018-06-29,2018-06-28,2018-06-27,2018-06-26,2018-06-25
count,289.000000,289,286.000000,284.000000,289.000000,289.000000,285.000000,289.000000,289.000000,289.000000,289.000000
mean,286.425606,0 days 11:58:22.422145,226.940559,210.042254,173.193772,170.096886,193.624561,208.570934,188.979239,229.865052,217.245675
std,98.151149,0 days 06:55:57.297439,87.680560,88.165751,65.434688,68.777603,91.536124,96.600602,84.186835,85.048102,84.347610
min,122.000000,0 days 00:00:00,43.000000,50.000000,41.000000,39.000000,47.000000,54.000000,58.000000,45.000000,52.000000
25%,207.000000,0 days 06:00:00,169.500000,146.750000,130.000000,107.000000,132.000000,126.000000,123.000000,190.000000,150.000000
50%,277.000000,0 days 12:00:00,241.500000,210.000000,175.000000,183.000000,179.000000,205.000000,174.000000,227.000000,232.000000
75%,352.000000,0 days 17:55:00,287.750000,258.250000,221.000000,231.000000,249.000000,294.000000,238.000000,283.000000,275.000000
max,543.000000,0 days 23:55:00,389.000000,397.000000,301.000000,299.000000,449.000000,447.000000,435.000000,407.000000,387.000000


In [17]:
conn = pymysql.connect(host='rr-wz9y03t9wtq075179o.mysql.rds.aliyuncs.com', user='medbuser', passwd='Ptmesys!!!1290098', db='mobility_exchange', port = 3306)
sql = 'SELECT TIME(FROM_TIME) as TIME,AVG_RUNNING, TRIPS_COUNT FROM me_transport_hub_minute_stat WHERE HUB_CODE="SZ_AIR_BA" AND DATE(FROM_TIME)="{}" ORDER BY CRT_ON;'

AVG_RUNNING = pd.DataFrame()
TRIPS_COUNT = pd.DataFrame()

for i in range(14):
    date_i = datetime.date.today() - datetime.timedelta(days=(i+1))
    sql_i = sql.format(date_i)
    df_i = pd.read_sql_query(sql_i, con=conn)
    avg_i = pd.DataFrame()
    trips_i = pd.DataFrame()
    if len(AVG_RUNNING.columns)==0:
        AVG_RUNNING[[date_i,"TIME"]] = df_i[["AVG_RUNNING","TIME"]]
        TRIPS_COUNT[[date_i,"TIME"]] = df_i[["TRIPS_COUNT","TIME"]]
    else:
        avg_i[[date_i, "TIME"]] = df_i[["AVG_RUNNING", "TIME"]]
        trips_i[[date_i, "TIME"]] = df_i[["TRIPS_COUNT", "TIME"]]
        AVG_RUNNING = pd.merge(AVG_RUNNING, avg_i[[date_i,"TIME"]], how="outer", on="TIME")
        TRIPS_COUNT = pd.merge(TRIPS_COUNT, avg_i[[date_i,"TIME"]], how="outer", on="TIME")
AVG_RUNNING.drop_duplicates(inplace=True)
TRIPS_COUNT.drop_duplicates(inplace=True)
AVG_RUNNING.reset_index(drop=True,inplace=True)
TRIPS_COUNT.reset_index(drop=True,inplace=True)
del AVG_RUNNING["TIME"]
del TRIPS_COUNT["TIME"]
AVG_RUNNING.to_csv("AVG_RUNNING_{}_created.csv".format(datetime.date.today()))
TRIPS_COUNT.to_csv("TRIPS_COUNT_{}_created.csv".format(datetime.date.today()))